In [2]:
import pandas as pd
df = pd.read_csv("./data/BruteForce/21-03-2024-3h/netflows.csv")
print(df.shape)
df.head()

(221552, 80)


,class,dst_port,protocol,timestamp,flow_duration,flow_byts_s,flow_pkts_s,fwd_pkts_s,bwd_pkts_s,tot_fwd_pkts,...,bwd_pkts_b_avg,fwd_blk_rate_avg,bwd_blk_rate_avg,fwd_seg_size_avg,bwd_seg_size_avg,cwe_flag_count,subflow_fwd_pkts,subflow_bwd_pkts,subflow_fwd_byts,subflow_bwd_byts
0,[0],34304,6,2024-03-21 07:22:30,14868.974686,28246.735829,403.524798,269.016532,134.508266,4,...,0.0,0.0,0.0,66.0,78.0,0,4,2,264,156
1,[0],58084,6,2024-03-21 07:22:30,14248.132706,34109.732836,491.292448,280.738542,210.553906,4,...,0.0,0.0,0.0,66.0,74.0,0,4,3,264,222
2,[0],58098,6,2024-03-21 07:22:30,14393.329620,29180.183535,416.859765,277.906510,138.953255,4,...,0.0,0.0,0.0,66.0,78.0,0,4,2,264,156
3,[0],58110,6,2024-03-21 07:22:31,13998.031616,34719.167189,500.070309,285.754462,214.315847,4,...,0.0,0.0,0.0,66.0,74.0,0,4,3,264,222
4,[0],57044,6,2024-03-21 07:22:31,14006.614685,34697.891741,499.763873,285.579356,214.184517,4,...,0.0,0.0,0.0,66.0,74.0,0,4,3,264,222


Time Ajustement

In [37]:
# df['timestamp'] = pd.to_datetime(df['timestamp']) - pd.to_timedelta(5, unit='h')
# df.head()

Time Filter using Info.txt

In [45]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

start_date = '2024-03-13 20:51:36'
end_date = '2024-03-13 22:04:48'

# Filter rows between start_date and end_date
filtered_df = df[(df['timestamp'] >= start_date) & (df['timestamp'] <= end_date)]
print(df.shape)
print(filtered_df.shape)
filtered_df['class'] = filtered_df['class'].replace({'[0]': 0, '[1]': 1})
filtered_df[filtered_df['class']==1]

(43897, 80)
(8081, 80)


C:\Users\hr_mo\AppData\Local\Temp\ipykernel_7472\978242981.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['class'] = filtered_df['class'].replace({'[0]': 0, '[1]': 1})


,class,dst_port,protocol,timestamp,flow_duration,flow_byts_s,flow_pkts_s,fwd_pkts_s,bwd_pkts_s,tot_fwd_pkts,...,bwd_pkts_b_avg,fwd_blk_rate_avg,bwd_blk_rate_avg,fwd_seg_size_avg,bwd_seg_size_avg,cwe_flag_count,subflow_fwd_pkts,subflow_bwd_pkts,subflow_fwd_byts,subflow_bwd_byts
956,1,80,6,2024-03-13 21:14:47,5.303760e+06,2.366246e+02,1.885455,1.131273,0.754182,6,...,0.0,0.0,0.0,90.166667,178.500000,0,6,4,541,714
1315,1,22,6,2024-03-13 21:26:12,6.631589e+04,8.459511e+03,120.634739,75.396712,45.238027,5,...,0.0,0.0,0.0,62.800000,82.333333,0,5,3,314,247
2179,1,80,6,2024-03-13 21:54:09,4.301399e+06,3.722044e+02,1.859860,1.162412,0.697447,5,...,0.0,0.0,0.0,146.600000,289.333333,0,5,3,733,868
2180,1,80,6,2024-03-13 21:54:09,4.300038e+06,4.225544e+02,1.860449,1.162780,0.697668,5,...,0.0,0.0,0.0,127.800000,392.666667,0,5,3,639,1178
2181,1,80,6,2024-03-13 21:54:09,4.300728e+06,3.555212e+02,1.860150,1.162594,0.697556,5,...,0.0,0.0,0.0,132.200000,289.333333,0,5,3,661,868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8217,1,80,6,2024-03-13 22:04:47,5.967617e+02,9.920208e+05,8378.553736,5027.132241,3351.421494,3,...,0.0,0.0,0.0,150.666667,70.000000,0,3,2,452,140
8218,1,80,6,2024-03-13 22:04:47,8.721352e+02,7.338312e+05,5733.056315,3439.833789,2293.222526,3,...,0.0,0.0,0.0,166.666667,70.000000,0,3,2,500,140
8219,1,80,6,2024-03-13 22:04:47,1.406908e+03,5.423240e+05,3553.892561,2132.335536,1421.557024,3,...,0.0,0.0,0.0,207.666667,70.000000,0,3,2,623,140
8220,1,80,6,2024-03-13 22:04:47,5.989075e+02,1.175474e+06,8348.535032,5009.121019,3339.414013,3,...,0.0,0.0,0.0,188.000000,70.000000,0,3,2,564,140


In [46]:
filtered_df.to_csv("./output/filtered_df.csv", index=False)

Read CPU data

In [47]:
df_cpu = pd.read_csv("./data/DOS(13-03-2024-17h)/CPU Usage-data-as-joinbyfield-2024-03-14 18_01_15.csv")
df_cpu.head()

,Time,Value 1,classifier,controller,database,sniffer
0,2024-03-13 20:51:36,0,0.0286,0.00673,0.00065,0.97
1,2024-03-13 20:51:38,0,0.0286,0.00673,0.00065,0.97
2,2024-03-13 20:51:40,0,0.0286,0.00673,0.00065,0.97
3,2024-03-13 20:51:42,0,0.0286,0.00673,0.00065,0.97
4,2024-03-13 20:51:44,0,0.0286,0.00673,0.00065,0.97


Data mapping

In [50]:

def data_mapping(date, df):
    df_date = df[df['timestamp'] == date]
    if not df_date.empty:
        df_date= df_date.drop('timestamp', axis=1)
        new_row = df_date.sum(numeric_only=True)
        new_row['nb_flows'] = len(df_date)
        new_row['nb_ports']= len(df_date['dst_port'].unique())
        new_row['nb_protocol']= len(df_date['protocol'].unique())
    else:
        new_row = df.iloc[0].apply(lambda x: 0)
        new_row['nb_flows'] = 0
        new_row['nb_ports']= 0
        new_row['nb_protocol']= 0

    return new_row
dfs_to_concat = []
final_df = pd.DataFrame()

for index, row in df_cpu.iterrows():
    new_row = data_mapping(row['Time'], filtered_df)
    last_row = pd.concat([new_row, row])
    dfs_to_concat.append(last_row)

final_df = pd.concat(dfs_to_concat, axis=1)
final_df = final_df.T
final_df = final_df.drop(['dst_port', 'protocol', 'timestamp'], axis=1)
final_df


,class,flow_duration,flow_byts_s,flow_pkts_s,fwd_pkts_s,bwd_pkts_s,tot_fwd_pkts,tot_bwd_pkts,totlen_fwd_pkts,totlen_bwd_pkts,...,subflow_bwd_byts,nb_flows,nb_ports,nb_protocol,Time,Value 1,classifier,controller,database,sniffer
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2024-03-13 20:51:36,0,0.0286,0.00673,0.00065,0.97
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2024-03-13 20:51:38,0,0.0286,0.00673,0.00065,0.97
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2024-03-13 20:51:40,0,0.0286,0.00673,0.00065,0.97
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2024-03-13 20:51:42,0,0.0286,0.00673,0.00065,0.97
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2024-03-13 20:51:44,0,0.0286,0.00673,0.00065,0.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2191,49.0,665354827.880859,620208488.108683,3082490.693186,1593832.192168,1488658.501018,576.0,303.0,151073.0,54332.0,...,54332.0,257.0,78.0,1.0,2024-03-13 22:04:40,0,0.0186,0.00389,0.000416,0.00832
2192,0.0,8639618.873596,1683.40073,4.629834,4.16685,0.462983,18.0,2.0,7124.0,148.0,...,148.0,2.0,1.0,1.0,2024-03-13 22:04:42,0,0.0186,0.00389,0.000416,0.00832
2193,0.0,4544154.167175,2393.373719,7.658655,6.747818,0.910836,15.0,2.0,5380.0,148.0,...,148.0,2.0,1.0,1.0,2024-03-13 22:04:44,0,0.0186,0.00389,0.000416,0.00832
2194,2.0,2051138.877869,1675.161778,11.700818,7.800545,3.900273,8.0,4.0,1438.0,280.0,...,280.0,2.0,1.0,1.0,2024-03-13 22:04:46,0,0.0186,0.00389,0.000416,0.00832


In [52]:
single_value_cols = [col for col in final_df.columns if final_df[col].nunique()<10]
single_value_cols.remove('nb_protocol')
print(single_value_cols)
final_df.drop(columns=single_value_cols, inplace=True)

['fwd_psh_flags', 'bwd_psh_flags', 'fwd_urg_flags', 'bwd_urg_flags', 'syn_flag_cnt', 'rst_flag_cnt', 'psh_flag_cnt', 'ack_flag_cnt', 'urg_flag_cnt', 'ece_flag_cnt', 'cwe_flag_count', 'Value 1']


In [53]:
from datetime import datetime
final_df['Time'] = pd.to_datetime(final_df['Time'])
now = pd.Timestamp(datetime.now())
final_df['seconds_from_now'] = (now - final_df['Time']).dt.total_seconds()
final_df.to_csv("./output/df_cpu_cleaned.csv", index=False)